In [1]:
import chess.pgn
import pandas as pd
import chess
import numpy as np
from scipy import linalg

In [2]:
ls -shr

total 90M
 16K README.md                              572K chess.ipynb
 89M lichess_db_standard_rated_2013-01.pgn   24K chess-and-eigenvectors.ipynb


In [3]:
board = chess.Board()

In [4]:
pgn = open('lichess_db_standard_rated_2013-01.pgn',encoding="utf-8")

In [5]:
#creates a list with one entry for each space on the board
spaces = []
for i in range(1,9):
    for j in range (1,9):
        spaces.append(chr(j+96)+str(i))

In [6]:
#create a matrix to tally weights for every pair of opposing pieces
pieces_list = [space for space in spaces if any(map(space.__contains__, ['1','2','7','8']))]
weights = pd.DataFrame(0,index=pieces_list, columns=pieces_list)

In [7]:
#create a state dictionary of each square and the initial square of each piece
#Those 'None' entries are the empty squares in the middle of the board, before the first move
#Each piece is on it's own square to begin
state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}
state_dict

{'a1': 'a1',
 'b1': 'b1',
 'c1': 'c1',
 'd1': 'd1',
 'e1': 'e1',
 'f1': 'f1',
 'g1': 'g1',
 'h1': 'h1',
 'a2': 'a2',
 'b2': 'b2',
 'c2': 'c2',
 'd2': 'd2',
 'e2': 'e2',
 'f2': 'f2',
 'g2': 'g2',
 'h2': 'h2',
 'a3': None,
 'b3': None,
 'c3': None,
 'd3': None,
 'e3': None,
 'f3': None,
 'g3': None,
 'h3': None,
 'a4': None,
 'b4': None,
 'c4': None,
 'd4': None,
 'e4': None,
 'f4': None,
 'g4': None,
 'h4': None,
 'a5': None,
 'b5': None,
 'c5': None,
 'd5': None,
 'e5': None,
 'f5': None,
 'g5': None,
 'h5': None,
 'a6': None,
 'b6': None,
 'c6': None,
 'd6': None,
 'e6': None,
 'f6': None,
 'g6': None,
 'h6': None,
 'a7': 'a7',
 'b7': 'b7',
 'c7': 'c7',
 'd7': 'd7',
 'e7': 'e7',
 'f7': 'f7',
 'g7': 'g7',
 'h7': 'h7',
 'a8': 'a8',
 'b8': 'b8',
 'c8': 'c8',
 'd8': 'd8',
 'e8': 'e8',
 'f8': 'f8',
 'g8': 'g8',
 'h8': 'h8'}

In [8]:
game_counter = 0
total_move_counter = 0
game = chess.pgn.read_game(pgn)

while game is not None:
    counter = 0
    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

    #print(game)
    #do stuff here



    #for each line in the file
    for move in game.mainline_moves():
        counter += 1
        if board.is_pseudo_legal(move):


            if board.is_capture(move):
                capturing_piece = state_dict[chess.square_name(move.from_square)]
                captured_piece = state_dict[chess.square_name(move.to_square)]
                #print('capturing piece',capturing_piece)
                #print('captured piece',captured_piece)

                if board.is_en_passant(move):
                    #do en passant stuff here
                    #print("en passant")
                    pass
                elif capturing_piece is not None:
                    if captured_piece is not None:

                        #record the kill in a matrix of all kills
                        #print(counter,chess.square_name(move.from_square), "takes on",chess.square_name(move.to_square))
                        #print(counter,state_dict[chess.square_name(move.from_square)], "takes",state_dict[chess.square_name(move.to_square)])

                        #what piece is there now?
                     

                        #increment an entry in the weights matrix when one piece takes another
                        weights.loc[state_dict[chess.square_name(move.from_square)],state_dict[chess.square_name(move.to_square)]] += 1

                        #update the piece on the target square
                        state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                        #update the source square to be empty
                        state_dict[chess.square_name(move.from_square)] = None

                        #board.push(move)  # Make the move


                        #if df.loc[i+1,'move_no'] == 1:
                        #    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

            else:


                #add castling special cases to record both moves
                if board.is_kingside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles kingside')
                        #move the appopriate rook
                        state_dict['f1'] = 'h1'
                        #leaving the rook source square empty
                        state_dict['h1'] = 'None'
                        #board.push(move)  # Make the move



                    elif not board.turn:
                        #print(counter,'... and black castles kingside')
                        #move the appopriate rook
                        state_dict['f8'] = 'h8'
                        #leaving the rook source square empty
                        state_dict['h8'] = 'None'
                        #board.push(move)  # Make the move

                elif board.is_queenside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles queenside')
                        #move the appopriate rook
                        state_dict['d1'] = 'a1'
                        #leaving the rook source square empty
                        state_dict['a1'] = 'None'
                        #board.push(move)  # Make the move

                    elif not board.turn:
                        #print(counter,'... and black castles queenside')
                        #move the appopriate rook
                        state_dict['d8'] = 'a8'
                        #leaving the rook source square empty
                        state_dict['a8'] = 'None'
                        #board.push(move)  # Make the move

                else:
                    #update the piece on the target square
                    state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                    #update the source square to be empty
                    state_dict[chess.square_name(move.from_square)] = None


                    #print(counter, chess.square_name(move.from_square),chess.square_name(move.to_square))

        board.push(move)  # Make the move
        #print(state_dict)
            
    if board.is_checkmate():
        checkers_list = [state_dict[chess.square_name(x)] for x in list(board.checkers())]
        if checkers_list[0] is not None:
            piece_giving_check = str(checkers_list[0])

            if board.turn:
                captured_king = 'e1'
            else:
                captured_king = 'e8'
            weights.loc[piece_giving_check,captured_king] += 1

    
    game_counter += 1
    #print('end of game',game_counter)
    
    total_move_counter += counter
    #print('total moves analyzed',total_move_counter)

    
    #print('board outcome',board.outcome())


    #print('is game over?',board.is_game_over())
    board.reset()

    game = chess.pgn.read_game(pgn)




    #print(state_dict)

    

KeyboardInterrupt: 

In [9]:
print(game_counter,'games analyzed')
print(total_move_counter,'moves analyzed')

6165 games analyzed
404833 moves analyzed


In [10]:
weights

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,g7,h7,a8,b8,c8,d8,e8,f8,g8,h8
a1,0,3,3,3,0,0,1,5,12,8,...,142,149,611,194,210,252,107,224,158,471
b1,2,0,3,1,0,2,1,3,5,3,...,83,70,189,256,329,240,16,485,651,151
c1,7,6,0,5,0,4,3,2,9,5,...,210,161,133,484,0,201,15,1049,1021,264
d1,4,3,0,0,1,3,2,3,5,4,...,352,339,425,659,795,1341,472,603,596,415
e1,0,0,0,0,0,0,0,0,0,0,...,22,38,42,52,60,134,0,86,76,54
f1,4,4,2,3,0,0,3,6,1,2,...,91,151,237,931,1050,167,22,0,530,140
g1,6,3,4,1,0,1,0,5,2,4,...,156,126,115,783,427,217,16,326,282,186
h1,3,4,3,5,1,3,2,0,5,5,...,201,201,504,192,263,282,102,280,222,576
a2,1,0,1,1,0,0,1,1,0,2,...,6,7,22,76,50,27,17,31,17,14
b2,2,2,1,1,0,0,1,1,6,0,...,3,0,23,80,28,26,9,364,105,6


In [11]:
#subtracting the identity matrix is necessary, forming an implicit set of homogeneous equations
a = weights - np.identity(32)
#print(a)

In [12]:
b = np.zeros(32)
#print(b)

In [13]:
x = linalg.solve(a, b)
x

array([ 0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
       -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
       -0., -0., -0., -0., -0.,  0.])

### Why doesn't it work?
### Because the solver is finding the trivial solution. i.e. just a vector full of zeroes.
### To check that there is actually a non trivial solution, the determinant of A must be non-zero.

In [14]:
np.linalg.det(a)

-3.425514479583275e+76

### OK since the determinant is not zero, there must be at least one non-trivial solution. How to find it?
### An homogenous linear equation can be solved by finding the eigenvectors.

In [15]:
eigen_values, eigen_vectors = np.linalg.eig(a)  
eigen_vectors

array([[-0.17056758+0.j,  0.17288505+0.j,  0.22811419+0.j, ...,
         0.01836041+0.j, -0.04976581+0.j, -0.04292834+0.j],
       [-0.19555023+0.j,  0.19542711+0.j,  0.03707325+0.j, ...,
         0.04443487+0.j,  0.00987583+0.j,  0.0095468 +0.j],
       [-0.23946289+0.j,  0.24188616+0.j,  0.17945771+0.j, ...,
         0.01173443+0.j, -0.0260924 +0.j, -0.02474098+0.j],
       ...,
       [ 0.24297495+0.j,  0.24389296+0.j, -0.1837127 +0.j, ...,
         0.04011321+0.j, -0.03845785+0.j,  0.0193254 +0.j],
       [ 0.19654534+0.j,  0.19686363+0.j, -0.00488623+0.j, ...,
        -0.00665908+0.j,  0.02045793+0.j, -0.01530125+0.j],
       [ 0.17569117+0.j,  0.17779904+0.j, -0.22118628+0.j, ...,
         0.11031072+0.j,  0.04958539+0.j,  0.00077347+0.j]])

In [16]:
values = eigen_vectors[:, np.argmax(eigen_values)] 
values
#Notice that the entries of the eigenvector corresponding to the largest eigenvalue are all real valued,
#So it's OK to throw away the imaginary component

array([0.17288505+0.j, 0.19542711+0.j, 0.24188616+0.j, 0.46950369+0.j,
       0.04569329+0.j, 0.23542606+0.j, 0.20351791+0.j, 0.18946656+0.j,
       0.02901248+0.j, 0.05916397+0.j, 0.05896607+0.j, 0.07153585+0.j,
       0.07876993+0.j, 0.06159002+0.j, 0.05238271+0.j, 0.03037732+0.j,
       0.02453834+0.j, 0.06600377+0.j, 0.06818156+0.j, 0.07453653+0.j,
       0.08173348+0.j, 0.06222091+0.j, 0.0547451 +0.j, 0.03142274+0.j,
       0.182428  +0.j, 0.19523114+0.j, 0.22673131+0.j, 0.45391829+0.j,
       0.05612325+0.j, 0.24389296+0.j, 0.19686363+0.j, 0.17779904+0.j])

In [17]:
## Let's normalize the values


In [18]:
list = []
for i in values:
    list.append(abs(float(i)))

<ipython-input-18-fd238d3eb4fa>:3: ComplexWarning: Casting complex values to real discards the imaginary part
  list.append(abs(float(i)))


In [19]:
#The fractional inverse of the average of all the pawns
factor = 1 / np.mean(list[8:24])

In [20]:
normed = [i*factor for i in list]
#normed

In [21]:
normalized_list = ["%.2f"%i for i in normed]

In [22]:
print(normalized_list[24:32])
print(normalized_list[16:24])
print(normalized_list[8:16])
print(normalized_list[0:8])


['3.22', '3.45', '4.01', '8.02', '0.99', '4.31', '3.48', '3.14']
['0.43', '1.17', '1.21', '1.32', '1.44', '1.10', '0.97', '0.56']
['0.51', '1.05', '1.04', '1.26', '1.39', '1.09', '0.93', '0.54']
['3.06', '3.45', '4.28', '8.30', '0.81', '4.16', '3.60', '3.35']


In [23]:
#tempo difference
white_sum = sum(normed[0:16])
black_sum = sum(normed[16:32])


print('total value of white pieces is', white_sum)
print('total value of black pieces is', black_sum)
print('total difference in value between white and black pieces is', white_sum-black_sum)
#This implies that white's tempo advantage is worth about 3 thousandths of an average pawn.

total value of white pieces is 38.8095590614949
total value of black pieces is 38.82309670897744
total difference in value between white and black pieces is -0.013537647482536386


In [24]:
#Notice that, for each pair of opposing pawns, the black pawn is stronger in 7 out of 8 pairs
#Why should this be? Possibly because white has the initial tempo for attacking early in the game,
#giving the black pawns many opportunities for capturing pieces defensively, and practically necessitating it.

In [25]:
#find average strength of all the pieces taken by each piece
#eg, rook strength seems low in general, probably because they come out later in the game
#and have fewer opportunities to capture lots of pieces,
#but they probably capture, on average, stronger pieces, in proportion to their own strength 

In [26]:
#Try a Matrix Decomposition

In [27]:
U, D, V = np.linalg.svd(a, full_matrices=True)

In [28]:
D

array([3.74938878e+03, 3.66613264e+03, 1.35635454e+03, 1.34449388e+03,
       1.26721561e+03, 1.22685737e+03, 1.02984183e+03, 1.02070282e+03,
       8.40239584e+02, 7.90074703e+02, 7.35467243e+02, 6.59664364e+02,
       5.83252442e+02, 5.63502415e+02, 3.56424646e+02, 3.40807732e+02,
       3.08867045e+02, 2.98593447e+02, 2.05098877e+02, 1.98615794e+02,
       1.81634020e+02, 1.69745053e+02, 1.60271710e+02, 1.39481172e+02,
       1.12939391e+02, 7.82988813e+01, 6.72241125e+01, 5.87504378e+01,
       5.05270734e+01, 3.98045388e+01, 6.65447754e+00, 5.37303825e-02])